In [1]:
import json

import matplotlib.pyplot as plt
import osmnx as ox 
import numpy as np

In [2]:
with open('./data/matched-results-1500.json', 'r') as f:
    match_results = json.load(f)
print(len(match_results))

match_results = [r for r in match_results if r['MATCHED_RESULTS'].get('Matched_path')]
print(len(match_results))

1500
1087


In [3]:
G = ox.io.load_graphml(filepath="./data/map/porto.graphml")
fig, ax = ox.plot_graph(G, node_size=3, figsize=(40, 40), edge_linewidth=1, show=False)

colors = plt.cm.jet(np.linspace(0, 1, 15))  
for i in range(15):
    gps_points = match_results[i]['POLYLINE']

    mgeom = [[float(j) for j in i.split(' ')] for i in match_results[i]['MATCHED_RESULTS']['Matched_geometry'][11:-1].split(',')]
    ax.plot([i[0] for i in mgeom], [i[1] for i in mgeom], color=colors[i][:-1], linewidth=1.25)
    
    x, y = zip(*gps_points)
    ax.scatter(x, y, color=colors[i][:-1], marker='x', s=15)

fig.savefig('./data/matched-15-routes.png', dpi=320)
plt.show()